In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType

from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main():
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		config.dataset.test,
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=config.data_loader.batch_size,
		shuffle=config.data_loader.shuffle)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel().to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p2d = test_batch['fisheye_joints_2d']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['regressor_dict']['pred_joint']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%300==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(config.eval.output_file, res)

	LOGGER.info('Done.')

In [3]:
main()

11:12:02 [INFO] Main: Starting demo...
  0%|          | 0/1807 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
 17%|█▋        | 300/1807 [23:06<1:53:27,  4.52s/it]

{'FullBody': {'All': 112.56929779052734, 'UpperStretching': 122.64324951171875, 'Walking': 113.20113372802734, 'Greeting': 86.53775787353516, 'LowerStretching': 122.21743774414062, 'Gaming': 84.30856323242188, 'Reacting': 81.01461029052734, 'Gesticuling': 70.42829132080078, 'Talking': 71.8091049194336, 'Patting': 88.69086456298828}, 'UpperBody': {'All': 81.37679290771484, 'UpperStretching': 91.99188995361328, 'Walking': 76.277587890625, 'Greeting': 66.19696044921875, 'LowerStretching': 72.29322052001953, 'Gaming': 66.71733856201172, 'Reacting': 66.5262451171875, 'Gesticuling': 63.23921203613281, 'Talking': 59.3539924621582, 'Patting': 61.31161880493164}, 'LowerBody': {'All': 143.76181030273438, 'UpperStretching': 153.29461669921875, 'Walking': 150.1246795654297, 'Greeting': 106.87852478027344, 'LowerStretching': 172.1416473388672, 'Gaming': 101.89979553222656, 'Reacting': 95.50296783447266, 'Gesticuling': 77.61736297607422, 'Talking': 84.26421356201172, 'Patting': 116.07010650634766}}


 33%|███▎      | 600/1807 [46:07<1:33:15,  4.64s/it]

{'FullBody': {'All': 112.74884033203125, 'UpperStretching': 122.05651092529297, 'Walking': 116.36000061035156, 'Greeting': 82.28779602050781, 'LowerStretching': 123.94261932373047, 'Gaming': 82.7070083618164, 'Reacting': 80.7503662109375, 'Gesticuling': 69.98396301269531, 'Talking': 69.75936889648438, 'Patting': 90.05760955810547}, 'UpperBody': {'All': 81.2941665649414, 'UpperStretching': 91.59918212890625, 'Walking': 77.49098205566406, 'Greeting': 64.36100769042969, 'LowerStretching': 72.55187225341797, 'Gaming': 65.18093872070312, 'Reacting': 66.32563781738281, 'Gesticuling': 62.85409164428711, 'Talking': 57.78786087036133, 'Patting': 62.80833435058594}, 'LowerBody': {'All': 144.20350646972656, 'UpperStretching': 152.51385498046875, 'Walking': 155.22903442382812, 'Greeting': 100.2145767211914, 'LowerStretching': 175.33335876464844, 'Gaming': 100.23307037353516, 'Reacting': 95.17509460449219, 'Gesticuling': 77.11383056640625, 'Talking': 81.73088073730469, 'Patting': 117.30687713623047

 50%|████▉     | 900/1807 [1:09:08<1:08:07,  4.51s/it]

...

 83%|████████▎ | 1500/1807 [1:55:03<23:42,  4.63s/it]

{'FullBody': {'All': 112.91996002197266, 'UpperStretching': 122.2750473022461, 'Walking': 115.5777816772461, 'Greeting': 84.12886810302734, 'LowerStretching': 123.71229553222656, 'Gaming': 83.79143524169922, 'Reacting': 81.9765396118164, 'Gesticuling': 70.78235626220703, 'Talking': 70.99785614013672, 'Patting': 88.22468566894531}, 'UpperBody': {'All': 81.28720092773438, 'UpperStretching': 91.53569793701172, 'Walking': 77.23231506347656, 'Greeting': 65.06771850585938, 'LowerStretching': 72.64281463623047, 'Gaming': 65.53607940673828, 'Reacting': 66.76271057128906, 'Gesticuling': 63.418052673339844, 'Talking': 57.75278854370117, 'Patting': 61.76637268066406}, 'LowerBody': {'All': 144.55271911621094, 'UpperStretching': 153.01438903808594, 'Walking': 153.92323303222656, 'Greeting': 103.19001007080078, 'LowerStretching': 174.78176879882812, 'Gaming': 102.04678344726562, 'Reacting': 97.19038391113281, 'Gesticuling': 78.14665985107422, 'Talking': 84.24291229248047, 'Patting': 114.682991027832

100%|█████████▉| 1800/1807 [2:18:05<00:31,  4.55s/it]

{'FullBody': {'All': 113.01914978027344, 'UpperStretching': 122.31704711914062, 'Walking': 115.83509063720703, 'Greeting': 84.69815063476562, 'LowerStretching': 123.754150390625, 'Gaming': 83.71766662597656, 'Reacting': 81.97964477539062, 'Gesticuling': 71.06593322753906, 'Talking': 71.2071762084961, 'Patting': 88.28023529052734}, 'UpperBody': {'All': 81.33580780029297, 'UpperStretching': 91.49663543701172, 'Walking': 77.5543212890625, 'Greeting': 65.16535949707031, 'LowerStretching': 72.68687438964844, 'Gaming': 65.33079528808594, 'Reacting': 66.83357238769531, 'Gesticuling': 63.585166931152344, 'Talking': 57.89302444458008, 'Patting': 61.759979248046875}, 'LowerBody': {'All': 144.7024688720703, 'UpperStretching': 153.13746643066406, 'Walking': 154.11585998535156, 'Greeting': 104.23092651367188, 'LowerStretching': 174.82142639160156, 'Gaming': 102.10453033447266, 'Reacting': 97.12571716308594, 'Gesticuling': 78.54669189453125, 'Talking': 84.52133178710938, 'Patting': 114.8004913330078

100%|██████████| 1807/1807 [2:18:36<00:00,  4.60s/it]
13:30:40 [INFO] Main: Saving evaluation results...
13:30:40 [INFO] Main: Done.
